# Network Analysis for Time Series

TimeSmith includes powerful network analysis tools that convert time series into graphs, enabling novel analysis methods.

## What Makes This Amazing?

- **Visibility Graphs**: Convert time series to graphs using visibility algorithms
- **Recurrence Networks**: Find patterns through recurrence analysis
- **Transfer Entropy**: Measure causal relationships between series
- **Network Metrics**: Extract features from time series graphs
- **Multiscale Analysis**: Analyze patterns at different time scales

This is a unique feature that sets TimeSmith apart!


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

from timesmith.network import (
    HVGFeaturizer,
    NVGFeaturizer,
    RecurrenceNetworkFeaturizer,
    graph_summary,
    network_metrics,
    transfer_entropy,
)

np.random.seed(42)
print("Network analysis tools loaded!")


## 1. Horizontal Visibility Graph (HVG)

Convert a time series into a graph where nodes are data points and edges represent horizontal visibility.


In [ ]:
# Create a time series with interesting structure
dates = pd.date_range("2020-01-01", periods=50, freq="D")
y = pd.Series(
    np.sin(np.linspace(0, 4*np.pi, 50)) + 
    np.random.normal(0, 0.2, 50),
    index=dates
)

# Create HVG
hvg_featurizer = HVGFeaturizer()
hvg_featurizer.fit(y)
graph = hvg_featurizer.transform(y)

print(f"HVG created:")
print(f"  Nodes: {graph.number_of_nodes()}")
print(f"  Edges: {graph.number_of_edges()}")
print(f"  Density: {nx.density(graph):.4f}")

# Get network metrics
metrics = network_metrics(graph)
print(f"\nNetwork Metrics:")
for key, value in metrics.items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:.4f}")

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot time series
axes[0].plot(y.index, y.values, 'o-', linewidth=2, markersize=4)
axes[0].set_title("Original Time Series", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Date")
axes[0].set_ylabel("Value")
axes[0].grid(True, alpha=0.3)

# Plot network (simplified layout)
pos = nx.spring_layout(graph, k=0.5, iterations=50)
nx.draw(graph, pos, ax=axes[1], node_size=50, node_color="steelblue", 
        edge_color="gray", alpha=0.6, with_labels=False)
axes[1].set_title("Horizontal Visibility Graph", fontsize=14, fontweight="bold")
axes[1].axis("off")

plt.tight_layout()
plt.show()


## 2. Natural Visibility Graph (NVG)

Similar to HVG but uses natural visibility (Euclidean distance in time-value space).


In [ ]:
# Create NVG
nvg_featurizer = NVGFeaturizer()
nvg_featurizer.fit(y)
nvg_graph = nvg_featurizer.transform(y)

print(f"NVG created:")
print(f"  Nodes: {nvg_graph.number_of_nodes()}")
print(f"  Edges: {nvg_graph.number_of_edges()}")
print(f"  Density: {nx.density(nvg_graph):.4f}")

# Compare HVG vs NVG
print(f"\nComparison:")
print(f"  HVG edges: {graph.number_of_edges()}")
print(f"  NVG edges: {nvg_graph.number_of_edges()}")
print(f"  Difference: {abs(graph.number_of_edges() - nvg_graph.number_of_edges())}")

# Get graph summary
summary = graph_summary(nvg_graph)
print(f"\nNVG Summary:")
for key, value in summary.items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:.4f}")


## 3. Recurrence Networks

Find recurring patterns in time series using recurrence networks.


In [ ]:
# Create recurrence network
recurrence_featurizer = RecurrenceNetworkFeaturizer(
    threshold=None,  # Auto-determine
    embedding_dimension=3,
    time_delay=1
)
recurrence_featurizer.fit(y)
recurrence_graph = recurrence_featurizer.transform(y)

print(f"Recurrence Network created:")
print(f"  Nodes: {recurrence_graph.number_of_nodes()}")
print(f"  Edges: {recurrence_graph.number_of_edges()}")
print(f"  Density: {nx.density(recurrence_graph):.4f}")

# Analyze network structure
if recurrence_graph.number_of_nodes() > 0:
    # Clustering coefficient
    clustering = nx.average_clustering(recurrence_graph)
    print(f"  Average clustering: {clustering:.4f}")
    
    # Path length
    if nx.is_connected(recurrence_graph):
        path_length = nx.average_shortest_path_length(recurrence_graph)
        print(f"  Average path length: {path_length:.4f}")
    else:
        print(f"  Graph is not connected (has {nx.number_connected_components(recurrence_graph)} components)")


## 4. Transfer Entropy: Causal Relationships

Measure information flow between time series to detect causal relationships.


In [ ]:
# Create two related time series
n = 100
x1 = np.sin(np.linspace(0, 4*np.pi, n)) + np.random.normal(0, 0.1, n)
x2 = np.roll(x1, 2) + np.random.normal(0, 0.1, n)  # x2 is a lagged version of x1

# Compute transfer entropy
try:
    te_1_to_2 = transfer_entropy(x1, x2, lag=1, bins=5)
    te_2_to_1 = transfer_entropy(x2, x1, lag=1, bins=5)
    
    print("Transfer Entropy Analysis:")
    print("=" * 50)
    print(f"TE(x1 → x2): {te_1_to_2:.4f}")
    print(f"TE(x2 → x1): {te_2_to_1:.4f}")
    print(f"\nInterpretation:")
    if te_1_to_2 > te_2_to_1:
        print(f"  x1 influences x2 more strongly (difference: {te_1_to_2 - te_2_to_1:.4f})")
    else:
        print(f"  x2 influences x1 more strongly (difference: {te_2_to_1 - te_1_to_2:.4f})")
except Exception as e:
    print(f"Transfer entropy computation: {e}")
    print("(This may require additional dependencies)")


## 5. Network Features as Time Series Features

Use network metrics as features for machine learning.


In [ ]:
# Extract network features from time series
hvg_featurizer = HVGFeaturizer()
hvg_featurizer.fit(y)
features = hvg_featurizer.transform(y)

print("Network Features Extracted:")
print(f"  Feature shape: {features.shape}")
print(f"  Feature columns: {list(features.columns)}")
print(f"\nFirst few rows:")
print(features.head())

# These features can now be used in machine learning pipelines!
print("\n✓ Network features ready for ML pipelines!")


## Summary

You've explored TimeSmith's unique network analysis capabilities:
- ✅ **Visibility Graphs**: Convert time series to graphs
- ✅ **Recurrence Networks**: Find patterns through recurrence
- ✅ **Transfer Entropy**: Measure causal relationships
- ✅ **Network Metrics**: Extract graph-based features
- ✅ **Feature Engineering**: Use network features in ML pipelines

**Why This Is Amazing:**
Network analysis provides a completely different perspective on time series data, enabling:
- Pattern detection through graph structure
- Causal inference between series
- Novel feature engineering approaches
- Multiscale analysis capabilities

This is a unique feature that makes TimeSmith stand out!


# Network Analysis for Time Series

This notebook demonstrates network-based analysis of time series.

## Creating Time Series

## Horizontal Visibility Graph

## Natural Visibility Graph

## Recurrence Network

## Transfer Entropy

## Windowed Network Analysis

## Multiscale Analysis